In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch_numopt
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, device="cpu"):
        super().__init__()
        self.f1 = nn.Linear(input_size, 10, device=device)
        self.f2 = nn.Linear(10, 20, device=device)
        self.f3 = nn.Linear(20, 20, device=device)
        self.f4 = nn.Linear(20, 10, device=device)
        self.f5 = nn.Linear(10, 1, device=device)

        self.activation = nn.ReLU()
        # self.activation = nn.Sigmoid()

    def forward(self, x):
        x = self.activation(self.f1(x))
        x = self.activation(self.f2(x))
        x = self.activation(self.f3(x))
        x = self.activation(self.f4(x))
        x = self.f5(x)

        return x

In [4]:
# X, y = load_diabetes(return_X_y=True, scaled=False)
# X, y = make_regression(n_samples=1000, n_features=100)
X, y = make_friedman1(n_samples=1000, noise=1e-2)

X_scaler = MinMaxScaler()
X = X_scaler.fit_transform(X)
X = torch.Tensor(X).to(device)

y_scaler = MinMaxScaler()
y = y_scaler.fit_transform(y.reshape((-1, 1)))
y = torch.Tensor(y).to(device)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch_data = TensorDataset(X_train, y_train)
data_loader = DataLoader(torch_data, batch_size=1000)

torch.Size([800, 10]) torch.Size([800, 1])
torch.Size([200, 10]) torch.Size([200, 1])


In [5]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=False,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

times = []

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0, loss: 0.218982994556427
epoch:  1, loss: 0.20870982110500336
epoch:  2, loss: 0.19838444888591766
epoch:  3, loss: 0.18788135051727295
epoch:  4, loss: 0.17770732939243317
epoch:  5, loss: 0.1675848513841629
epoch:  6, loss: 0.15772496163845062
epoch:  7, loss: 0.14778293669223785
epoch:  8, loss: 0.1381731927394867
epoch:  9, loss: 0.1289830505847931
epoch:  10, loss: 0.12015390396118164
epoch:  11, loss: 0.11167602986097336
epoch:  12, loss: 0.10355976223945618
epoch:  13, loss: 0.09589988738298416
epoch:  14, loss: 0.08874791115522385
epoch:  15, loss: 0.08205219358205795
epoch:  16, loss: 0.07606885582208633
epoch:  17, loss: 0.07027136534452438
epoch:  18, loss: 0.06508006155490875
epoch:  19, loss: 0.060216471552848816
epoch:  20, loss: 0.05584419146180153
epoch:  21, loss: 0.05175312981009483
epoch:  22, loss: 0.04785919189453125
epoch:  23, loss: 0.04424922913312912
epoch:  24, loss: 0.04080038145184517
epoch:  25, loss: 0.03757404908537865
epoch:  26, loss: 0.034718

In [6]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9900502405031074
Test metrics:  R2 = 0.9793472135066519


In [7]:
model = Net(input_size=X.shape[1], device=device)
loss_fn = nn.MSELoss()
opt = torch_numopt.LM(
    params=model.parameters(),
    model=model,
    lr=1,
    mu=0.001,
    mu_dec=0.1,
    use_diagonal=True,
    c1=1e-4,
    tau=0.1,
    line_search_method="backtrack",
    line_search_cond="armijo",
)

all_loss = {}
for epoch in range(100):
    print("epoch: ", epoch, end="")
    all_loss[epoch + 1] = 0
    for batch_idx, (b_x, b_y) in enumerate(data_loader):
        pre = model(b_x)
        loss = loss_fn(pre, b_y)
        opt.zero_grad()
        loss.backward()

        # parameter update step based on optimizer
        opt.step(b_x, b_y, loss_fn)

        all_loss[epoch + 1] += loss
    all_loss[epoch + 1] /= len(data_loader)
    print(", loss: {}".format(all_loss[epoch + 1].cpu().detach().numpy().item()))

epoch:  0

/home/eugeniolr/Documents/invAI/torch_numopt/src/torch_numopt/utils.py:33: UserWarning: torch.linalg.svd: During SVD computation with the selected cusolver driver, batches 0 failed to converge. A more accurate method will be used to compute the SVD as a fallback. Check doc at https://pytorch.org/docs/stable/generated/torch.linalg.svd.html (Triggered internally at /pytorch/aten/src/ATen/native/cuda/linalg/BatchLinearAlgebraLib.cpp:690.)
  U, S, Vt = torch.linalg.svd(mat)


, loss: 0.09479362517595291
epoch:  1, loss: 0.08853869885206223
epoch:  2, loss: 0.08481962233781815
epoch:  3, loss: 0.07500425726175308
epoch:  4, loss: 0.0682247132062912
epoch:  5, loss: 0.062481839209795
epoch:  6, loss: 0.05768628790974617
epoch:  7, loss: 0.04894156754016876
epoch:  8, loss: 0.0448191836476326
epoch:  9, loss: 0.04052858427166939
epoch:  10, loss: 0.037106938660144806
epoch:  11, loss: 0.03317578509449959
epoch:  12, loss: 0.029414257034659386
epoch:  13, loss: 0.025448020547628403
epoch:  14, loss: 0.022308597341179848
epoch:  15, loss: 0.01982753723859787
epoch:  16, loss: 0.017529431730508804
epoch:  17, loss: 0.0155929084867239
epoch:  18, loss: 0.01389321032911539
epoch:  19, loss: 0.01195856649428606
epoch:  20, loss: 0.010467266663908958
epoch:  21, loss: 0.009076748974621296
epoch:  22, loss: 0.0079647833481431
epoch:  23, loss: 0.007162810303270817
epoch:  24, loss: 0.00660259835422039
epoch:  25, loss: 0.006085315719246864
epoch:  26, loss: 0.00566984

In [8]:
pred_train = model.forward(X_train).cpu().detach()
pred_test = model.forward(X_test).cpu().detach()
print(f"Train metrics: R2 = {r2_score(pred_train, y_train.cpu())}")
print(f"Test metrics:  R2 = {r2_score(pred_test, y_test.cpu())}")

Train metrics: R2 = 0.9855669068830493
Test metrics:  R2 = 0.9816192728662396
